# Example code of Michaelis-Menten

ここでは、ミカエリスメンテン式に関するパラメータ推定を例として、私たちのコードでパラメータ推定を行う方法を解説する。


## どのような問題か  
ミカエリス–メンテン式に基づく酵素反応モデルを用いて、  
**実験（あるいは疑似）データから反応パラメータを推定する問題**を考える。

本問題では、

- 酵素反応の時間発展を常微分方程式（ODE）で記述し  
- 複数の初期基質濃度条件における観測データを用いて  
- モデルパラメータを統計的に推定する  

ことを目的とする。

---

### ミカエリスメンテン式とは

ミカエリス–メンテン式は、**酵素反応の速さが基質濃度によってどのように変化するか**を表す、最も基本的な反応速度式である。

酵素反応では、基質濃度が低い場合には基質を増やすほど反応速度はほぼ比例的に増加する。  
一方で、基質濃度が十分に高くなると酵素が飽和し、それ以上基質を増やしても反応速度はほとんど増加しなくなる。

この **「低濃度では比例、高濃度では飽和」** という性質を簡潔に表したものがミカエリス–メンテン式である。

---

### 数式表現

ミカエリス–メンテン式は以下のように表される。

$v = \dfrac{V_{\max}[S]}{K_m + [S]}$

ここで、

- $v$：反応速度  
- $[S]$：基質濃度  
- $V_{\max}$：最大反応速度  
- $K_m$：ミカエリス定数  

を表す。

---

### 時間発展モデルへの拡張

本問題では反応速度式を用いて、基質濃度の時間変化を次の常微分方程式で表す。

$\dfrac{dS(t)}{dt} = - \dfrac{V_{\max} S(t)}{K_m + S(t)}$

ここで $S(t)$ は時刻 $t$ における基質濃度である。

生成物量 $P(t)$ は、初期基質濃度 $S_0$ を用いて

$P(t) = S_0 - S(t)$

として計算される。

---

## 推定するパラメータについて

本問題で推定対象とするパラメータは以下である。

### 動力学パラメータ

- **最大反応速度 $V_{\max}$**  
  酵素が完全に飽和したときの反応速度を表す。

- **ミカエリス定数 $K_m$**  
  反応速度が $V_{\max}/2$ となるときの基質濃度であり、  
  酵素と基質の親和性の指標として解釈されることが多い。

### 観測ノイズパラメータ（必要に応じて）

- **観測誤差の標準偏差 $\sigma$**  
  観測値がモデル値からどの程度ばらつくかを表す。

解析条件に応じて、

- $\sigma$ を既知として固定する場合  
- $\sigma$ も未知パラメータとして同時に推定する場合  

の両方を考える。

---

## 疑似データの作成方法

実データの代わりに、既知の真のパラメータを用いて  
**疑似観測データ**を生成する。

### 疑似データ生成の手順

1. 真のパラメータ  
   $V_{\max}^{\mathrm{true}}, \; K_m^{\mathrm{true}}$
   を設定する。

2. 複数の初期基質濃度  
   $S_0^{(1)}, S_0^{(2)}, \dots, S_0^{(N)}$
   を用意する。

3. 各条件についてミカエリス–メンテン ODE を数値的に解き、  
   理論的な生成物量 $P_{\mathrm{true}}(t)$ を計算する。

4. 各時刻において、ガウス分布に従う観測ノイズを加える。

$P_{\mathrm{obs}}(t) = P_{\mathrm{true}}(t) + \varepsilon(t)$

$\varepsilon(t) \sim \mathcal{N}(0,\sigma^2)$

これにより、現実の実験データを模擬した疑似データを得る。

---

### 疑似データを用いる目的

- 推定アルゴリズムの妥当性検証  
- パラメータの同定可能性（identifiability）の確認  
- 推定精度および不確実性評価  

---

### まとめ

本問題は、

> ミカエリス–メンテン型酵素反応モデルを用いて、  
> 複数条件の時系列データから  
> 反応パラメータを統計的に推定する問題  

として定式化される。

この枠組みは、ベイズ推定・MCMC・SMC などの手法を検証する  
代表的なテスト問題として利用できる。


## 推定

本節では、ミカエリス–メンテンモデルに基づくパラメータ推定の設定と、  
その結果として得られる出力について説明する。

---

### 何をどう設定したのか

本問題では、以下のパラメータを推定対象とする。

- 最大反応速度 $V_{\max}$
- ミカエリス定数 $K_m$
- （必要に応じて）観測ノイズの標準偏差 $\sigma$

基質濃度の時間発展は、次の常微分方程式でモデル化される。

$\dfrac{dS(t)}{dt} = - \dfrac{V_{\max} S(t)}{K_m + S(t)}$

生成物量は

$P(t) = S_0 - S(t)$

として定義し、観測データは以下のガウスノイズモデルに従うと仮定する。

$P_{\mathrm{obs}}(t) = P_{\mathrm{model}}(t) + \varepsilon(t)$

$\varepsilon(t) \sim \mathcal{N}(0,\sigma^2)$

複数の初期基質濃度条件 $S_0^{(i)}$ に対して得られた時系列データを同時に用い、  
全条件で共通のパラメータ $(V_{\max}, K_m, \sigma)$ を推定する。

推定はベイズ的枠組みに基づき、尤度関数は次式で与えられる。

$\log p(\mathcal{D} \mid \theta)
= \sum_{i=1}^{N_{\mathrm{cond}}}
\sum_{j=1}^{N_t}
\left[
- \dfrac{1}{2}\log(2\pi\sigma^2)
- \dfrac{
\left(
P_{\mathrm{obs}}^{(i)}(t_j) - P_{\mathrm{model}}^{(i)}(t_j)
\right)^2
}{2\sigma^2}
\right]$

ここで $\theta = \{V_{\max}, K_m, \sigma\}$ は推定対象のパラメータ集合である。

---

以下のようにメインから呼び出す関数を定義した。

In [1]:
# --- ODEとシミュレーション（t配列を指定する版） ---
def mm_ode(t, S, Vmax, Km):
    return - Vmax * S / (Km + S)

def simulate_mm_on_grid(Vmax, Km, S0, t_array):
    """
    観測時刻 t_array に合わせてミカエリスメンテン進行曲線を解く。
    戻り値: P_model(t_array)
    """
    t_span = (t_array[0], t_array[-1])

    sol = solve_ivp(
        fun=lambda t, S: mm_ode(t, S, Vmax, Km),
        t_span=t_span,
        y0=[S0],
        t_eval=t_array,
        method="RK45",
    )
    S_model = sol.y[0]
    P_model = S0 - S_model
    return P_model

@ray.remote
def log_likelihood_mm_multi(params):
    """
    複数の S0 条件のデータを使った対数尤度関数。
    
    params  : [Vmax, Km, sigma]
    dataset : load_all_mm_data() で読み込んだデータのリスト
    """
        
    Vmax, Km, sigma = params

    # σを推定しない場合，sigmaはsigma_trueに固定される
    if est_sigma:
        sigma = params[-1]
    else:
        sigma = sigma_true


    if sigma <= 0:
        return -np.inf

    logL_total = 0.0
    P_model_list = []

    for i in range(n_ex):
        t     = dataset[i]["t"]
        P_obs = dataset[i]["P_obs"]
        S0    = dataset[i]["S0"]

        # モデル予測
        P_model = simulate_mm_on_grid(Vmax, Km, S0, t)

        # 残差
        residual = P_obs - P_model

        logL_i = -0.5 * datapoint * np.log(2*np.pi*sigma**2) \
                 - np.sum(residual**2) / (2*sigma**2)

        logL_total += logL_i
        P_model_list.append(P_model)


    return logL_total, P_model_list

def sim_particle(particle):
    print('sim_particle')

    # 並列化計算を実行する
    results = [log_likelihood_mm_multi.remote(particle[i,:]) for i in range(n_particle)]
    # print(results)

    # 並列計算の結果を取得する
    llk_Cl = ray.get(results)

    # 取得した結果を使える形に成型する
    llk, C_l_ = zip(*llk_Cl)

    return llk,C_l_


NameError: name 'ray' is not defined

### 出力

推定の結果として、以下の量が得られる。

- 各パラメータ $V_{\max}, K_m, (\sigma)$ の推定値  
- パラメータの事後分布（またはその近似）  
- 各粒子あるいはサンプルに対応するモデル予測 $P_{\mathrm{model}}(t)$  

これらの出力を用いることで、

- 推定値と真値（疑似データの場合）の比較  
- パラメータ不確実性の評価  
- モデル予測と観測データの整合性の確認  

が可能となる。

特に、異なる初期基質濃度条件に対して  
単一のパラメータ集合で一貫した説明が可能かどうかは、  
モデルの妥当性を判断する重要な指標となる。

## 考察

### 推定結果について

### 他の手法には適用可能か

\

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

## Markdown + notebooks

As it is markdown, you can embed images, HTML, etc into your posts!

![](https://myst-parser.readthedocs.io/en/latest/_static/logo-wide.svg)

You can also $add_{math}$ and

$$
math^{blocks}
$$

or

$$
\begin{aligned}
\mbox{mean} la_{tex} \\ \\
math blocks
\end{aligned}
$$

But make sure you \$Escape \$your \$dollar signs \$you want to keep!

## MyST markdown

MyST markdown works in Jupyter Notebooks as well. For more information about MyST markdown, check
out [the MyST guide in Jupyter Book](https://jupyterbook.org/content/myst.html),
or see [the MyST markdown documentation](https://myst-parser.readthedocs.io/en/latest/).

## Code blocks and outputs

Jupyter Book will also embed your code blocks and output in your book.
For example, here's some sample Matplotlib code:

In [ ]:
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
import numpy as np
plt.ion()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

N = 10
data = [np.logspace(0, 1, 100) + np.random.randn(100) + ii for ii in range(N)]
data = np.array(data).T
cmap = plt.cm.coolwarm
rcParams['axes.prop_cycle'] = cycler(color=cmap(np.linspace(0, 1, N)))


from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color=cmap(0.), lw=4),
                Line2D([0], [0], color=cmap(.5), lw=4),
                Line2D([0], [0], color=cmap(1.), lw=4)]

fig, ax = plt.subplots(figsize=(10, 5))
lines = ax.plot(data)
ax.legend(custom_lines, ['Cold', 'Medium', 'Hot']);

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)